# COMP5329 ASSIGNMENT 
## TRANSFER LEARN VGGNet16

### INSTRUCTIONS:
* Run cells in order to generate results

In [19]:
# --------------
# BASIC PACKAGES
# --------------
%matplotlib inline
import numpy as np
seed = 99
np.random.seed(99)
import matplotlib.pyplot as plt
import scipy as sp
import time
from datetime import timedelta
import math
import imageio
import os
import itertools

# ---------------------------
# TENSORFLOW & KERAS PACKAGES
# ---------------------------
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import model_from_json
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from sklearn.metrics import classification_report
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

# ------------------------------------
# SCIKIT-LEARN PACKAGES FOR EVALUATION
# ------------------------------------
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

### Define Image Import Function

In [6]:
def READ_IN_IMGS(X_train_folder, X_labels, img_height, img_width):
    
    start = time.time()
    no_of_samples = X_labels.shape[0]
    X_train_array = np.zeros((no_of_samples,img_height,img_width))
    j=0
    for i in X_labels:
        img_ary = imageio.imread(X_train_folder+i)
        img_ary = img_ary.reshape((1,) + img_ary.shape) 
        X_train_array[j] = img_ary
        if j < X_train_array.shape[0]:
            j+=1
    X_train_array = X_train_array.astype('float32')
    X_train_array = X_train_array / 255
    print(time.time() - start)
    
    return(X_train_array)

### Listdir for filenames

In [7]:
train_lbls = np.loadtxt('train.txt', dtype=str, delimiter=' ')
# (6262, 2)
vali_lbls = np.loadtxt('vali.txt', dtype=str, delimiter=' ')

In [8]:
np.random.seed(seed)
np.random.shuffle(train_lbls)
np.random.seed(seed)
np.random.shuffle(vali_lbls)

In [9]:
def READ_IN_IMGS(X_train_folder, X_labels, img_height, img_width):
    
    start = time.time()
    no_of_samples = X_labels.shape[0]
    X_train_array = np.zeros((no_of_samples,img_height,img_width))
    j=0
    for i in X_labels:
        img_ary = imageio.imread(X_train_folder+i)
        img_ary = img_ary.reshape((1,) + img_ary.shape) 
        X_train_array[j] = img_ary
        if j < X_train_array.shape[0]:
            j+=1
    X_train_array = X_train_array.astype('float32')
    X_train_array = X_train_array / 255
    print(time.time() - start)
    
    return(X_train_array)

In [10]:
X_train = READ_IN_IMGS('train-set/',train_lbls[:,0],128,128)
X_vali = READ_IN_IMGS('vali-set/',vali_lbls[:,0],128,128)

49.68698072433472
7.523626804351807


In [13]:
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)
X_vali = X_vali.reshape(X_vali.shape[0], 128, 128, 1)

In [20]:
X_train.shape

(37882, 128, 128, 1)

In [15]:
y_vali = np_utils.to_categorical(vali_lbls[:,1], 62)
y_train = np_utils.to_categorical(train_lbls[:,1], 62)
class_num = y_vali.shape[1]

In [60]:
ch3_Xtrain = np.stack((X_train, X_train, X_train),axis=3)

In [59]:
ch3_Xtrain.shape

(37882, 128, 128, 1, 3)

### VGGNet Transfer Learning Model:  Retraining only Fully Connected Layers
Made with refernce to: https://www.analyticsvidhya.com/blog/2017/06/transfer-learning-the-art-of-fine-tuning-a-pre-trained-model/

In [68]:
VVmodel = VGG16(weights='imagenet', include_top=False, input_shape=(128,128,3))

In [69]:
# 63 mins runtime on local machine
feat_train = model.predict(ch3_Xtrain[:,:,:,:,0])

In [77]:
21 mins runtime on local machine
feat_test=model.predict(ch3_Xtest[:,:,:,:,0])

In [83]:
feat_train.shape

(37882, 4, 4, 512)

In [84]:
train_X = feat_train.reshape(37882,8192)

In [87]:
model=Sequential()
model.add(Dense(1000, input_dim=8192, activation='relu',kernel_initializer='uniform'))
model.add(BatchNormalization(momentum=0.6))
keras.layers.core.Dropout(0.5)

model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(62))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [88]:
tfr_lrn = model.fit(train_X,y_train,batch_size=32,epochs=20,validation_split=0.1)

Train on 34093 samples, validate on 3789 samples
Epoch 1/20
34093/34093 [==============================] - 107s 3ms/step - loss: 1.4056 - acc: 0.7057 - val_loss: 0.5828 - val_acc: 0.8374
Epoch 2/20
34093/34093 [==============================] - 105s 3ms/step - loss: 0.8327 - acc: 0.7935 - val_loss: 0.4810 - val_acc: 0.8504
Epoch 3/20
34093/34093 [==============================] - 106s 3ms/step - loss: 0.6689 - acc: 0.8169 - val_loss: 0.4390 - val_acc: 0.8575
Epoch 4/20
34093/34093 [==============================] - 106s 3ms/step - loss: 0.5585 - acc: 0.8423 - val_loss: 0.4479 - val_acc: 0.8620
Epoch 5/20
34093/34093 [==============================] - 106s 3ms/step - loss: 0.4948 - acc: 0.8551 - val_loss: 0.3986 - val_acc: 0.8783
Epoch 6/20
34093/34093 [==============================] - 116s 3ms/step - loss: 0.4448 - acc: 0.8635 - val_loss: 0.3839 - val_acc: 0.8709
Epoch 7/20
34093/34093 [==============================] - 111s 3ms/step - loss: 0.4085 - acc: 0.8719 - val_loss: 0.3784 - v